In [2]:
import pyodbc

# Set variables for the project (i.e. the input location of the file to be processed and the output location) )
folderlink = '..//data//'
output_folder = 'output//'

def export_access_to_sql_dump(access_db_path, sql_dump_path):
    # Connect to the Access database
    conn_str = f'DRIVER={{Microsoft Access Driver (*.mdb, *.accdb)}};DBQ={access_db_path};'
    conn = pyodbc.connect(conn_str)
    cursor = conn.cursor()

    # Get the list of tables in the Access database
    tables = [row.table_name for row in cursor.tables(tableType='TABLE')]

    with open(sql_dump_path, 'w') as sql_file:
        for table in tables:
            # Write CREATE TABLE statement
            sql_file.write(f'-- Creating table {table}\n')
            sql_file.write(f'DROP TABLE IF EXISTS {table};\n')
            create_table_stmt = f'CREATE TABLE {table} (\n'
            cursor.execute(f'SELECT * FROM [{table}]')
            columns = [column[0] for column in cursor.description]
            column_types = []
            for column in columns:
                cursor.execute(f'SELECT TOP 1 [{column}] FROM [{table}]')
                row = cursor.fetchone()
                if row:
                    col_type = type(row[0]).__name__
                    if col_type == 'str':
                        col_type = 'TEXT'
                    elif col_type == 'int':
                        col_type = 'INTEGER'
                    elif col_type == 'float':
                        col_type = 'REAL'
                    elif col_type == 'datetime':
                        col_type = 'DATETIME'
                    else:
                        col_type = 'TEXT'
                else:
                    col_type = 'TEXT'
                column_types.append(col_type)
            columns_with_types = [f'[{columns[i]}] {column_types[i]}' for i in range(len(columns))]
            create_table_stmt += ',\n'.join(columns_with_types) + '\n);\n'
            sql_file.write(create_table_stmt)
            sql_file.write('\n')

            # Write INSERT INTO statements
            sql_file.write(f'-- Inserting data into {table}\n')
            cursor.execute(f'SELECT * FROM [{table}]')
            rows = cursor.fetchall()
            for row in rows:
                values = []
                for value in row:
                    if isinstance(value, str):
                        values.append("'{}'".format(value.replace("'", "''")))
                    elif value is None:
                        values.append('NULL')
                    else:
                        values.append(str(value))
                values_str = ', '.join(values)
                sql_file.write('INSERT INTO [{}] ({}) VALUES ({});\n'.format(table, ", ".join([f'[{col}]' for col in columns]), values_str))
            sql_file.write('\n')

    # Close the connection
    cursor.close()
    conn.close()

# Export CLERUS v1
access_db_path = 'E://digidure//CLERUS_v1_DRC_07082024.accdb'
sql_dump_path = folderlink+output_folder+'CLERUS_v1_DRC_07082024.sql'
export_access_to_sql_dump(access_db_path, sql_dump_path)